In [3]:
import torch
import torch.nn as nn
from torch.nn import functional as F

In [2]:
torch.manual_seed(1337);

In [3]:
# hyper_parameters
batch_size = 64
block_size = 256
eval_iters = 200
eval_interval = 500
max_iters = 5000
lr = 3e-4
n_emb = 384
n_head = 6
n_layer = 6
dropout = 0.2

In [8]:
with open("names.txt",'r') as f:
    text = f.read()

chars = sorted(list(set(text))) # we use set() as it eliminates duplicate characters
vocab_size = len(chars)

stoi = {ch:i for (i,ch) in enumerate(chars)}
itos = {i:ch for (ch,i) in stoi.items()}

encode = lambda txt: [stoi[c] for c in txt]
decode = lambda txt: ''.join([itos[i] for i in txt])

# encoding the entire dataset
data = torch.tensor(encode(text), dtype=torch.long)

In [4]:
# splitting dataset into train and val
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

In [5]:
def get_batch(split):
    # generate a small batch of data of inputs x and outputs y
    data = train_data if split == 'train' else val_data

    ix = torch.randint(len(data)-block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

# first input to transformer
xb, yb = get_batch('train')

In [17]:
def generate(model, max_new_tokens):
    # we kick off generation with '0' ie., newline(\n)
    # starting with one batch and one character
    return decode(model.generate(idx = torch.zeros((1,1), dtype=torch.long), max_new_tokens = max_new_tokens)[0].tolist())

In [18]:
@torch.no_grad()
def estimate_loss(): # this averages loss over mutiple batches and its lot less noisy
    out = {}
    m.eval()
    for split in ['train','val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X,Y = get_batch(split)
            loss, logits = m(X,Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    m.train()
    return out

In [7]:
# defining attention head
class Head(nn.Module):
    """ one head of self-attention """
    
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_emb, head_size, bias=False)
        self.query = nn.Linear(n_emb, head_size, bias=False)
        self.value = nn.Linear(n_emb, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)
    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x) # (B,T,C) where C -> head_size
        q = self.query(x) # (B,T,C) where C -> head_size

        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B,T,C) @ (B,C,T) -> (B,T,T)
        wei = wei.masked_fill(self.tril[:T,:T]==0, float('-inf')) # (B,T,T)
        wei  = F.softmax(wei, dim=-1) #(B,T,T)
        wei = self.dropout(wei)

        # perform weighted aggragation of the values
        v = self.value(x) # (B,T,C) where C -> head_size
        out = wei @ v # (B,T,T) @ (B,T,C) -> (B,T,C) where C -> head_size

        return out

In [8]:
# defining multihead-attention head
class MultiHeadAttention(nn.Module):

    def __init__(self,num_heads,head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_emb, n_emb) # this is the projection layer going back into the residual pathway
        self.dropout = nn.Dropout(dropout)
    
    def forward(self,x):
        # output of self-attentions
        out = torch.cat([h(x) for h in self.heads],dim=-1) # concatnating across channel dimension. we are feeding it into each self-attention heads
        out = self.dropout(self.proj(out))
        return out

In [9]:
class FeedForward(nn.Module):
    """ a simple linear layer followed by a non-linearity"""

    def __init__(self, n_emb):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_emb, n_emb),
            nn.ReLU(), 
            nn.Linear(n_emb, n_emb), # this is the projection layer going back into the residual pathway
            nn.Dropout(dropout)
        )
    
    def forward(self, x): # done on per token level. all the tokens do this independently
        return self.net(x)


In [10]:
# start to intersperse the communication with computation
# transformers does this when it has blocks that communicate and then computer, it groups them and replicates them
class Block(nn.Module):
    """ Transformer block: communication followed by computation"""

    def __init__(self, n_emb, n_head):
        # n_emb: emdedding dimension; n_head: the number of heads self-attention
        super().__init__()
        head_size = n_emb//n_head # we perform sel_attention to a equal part of the blocks
        self.sa = MultiHeadAttention(n_head, head_size) # performs computation between tokens
        self.ffwd = FeedForward(n_emb) # performs "thinking" on the tokens
        self.ln1 = nn.LayerNorm(n_emb)
        self.ln2 = nn.LayerNorm(n_emb) # make unit mean, unit gaussian at initializaion

    def forward(self, x):
        x = x + self.sa(self.ln1(x)) # we fork off some communication and we come back using addition(+)
        x = x + self.ffwd(self.ln2(x)) # we fork off some communication and we come back using addition(+)
        # these two above with plus(+)/addition(+) forms residual connections 
        return x

In [15]:
class BigramLanguageModel(nn.Module):
    def __init__(self):
        super().__init__()
        # each token dirctly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_emb) # to create level of interaction. we dont't dont directly go to embeddings for logits
        self.posistion_embedding_table = nn.Embedding(block_size, n_emb) # each postion from 0 to block_size-1 will also get some embeddig vector 
        self.blocks = nn.Sequential(*[Block(n_emb, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_emb) # final layer norm
        self.lm_head = nn.Linear(n_emb, vocab_size) # use linear layer to go from token_embeddings to logits. this layer decodes into vocabulary
        
    def forward(self, idx, targets=None):
        B,T = idx.shape
        
        # idx and targets are both targets of (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C). thic C -> n_emb
        pos_emb = self.posistion_embedding_table(torch.arange(T)) # this will be integers from 0 to T-1 and all these integers from 0 to T-1 get embedded through table to get (T,C), C -> n_emb. just like each token is mapped to the embedding table, each posistion is mapped using the posistion embedding table  
        x = tok_emb + pos_emb # (B,T,C). there will be broadcasting across batch. C -> n_emb. we add this postional embedding ie., posistions at which these tokens occur to every examples with the word embeddings. encoding information with token embeddings and posistional embeddings
        x = self.blocks(x)
        logits = self.lm_head(x) # (B,T,C). thic C -> vocab_size. output, 'x',from the previous step is fed into decoder language modelling head

        if targets is None:
            loss = None
        else:
            B,T,C = logits.shape
            logits = logits.view(B*T, C) # we are stretching along 0th direction ie., stretching each input idx to 1-D
            targets = targets.view(B*T) # we do this to match CrossEntropy loss of pytorch ie., 1D
            loss = F.cross_entropy(logits, targets)
        return loss,logits
    
    def generate(self, idx, max_new_tokens):
        # idx is (B,T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:,-block_size:] # '-block_size:' as we are taking token from last
            # get predictions
            loss, logits = self(idx_cond) # logits -> (B,T,C)
            # focus only on the last time step
            logits = logits[:,-1,:] # becomes (B, C). this line means that we are choosing the lateset generated character
            # apply softmax to get probabilities
            prob = F.softmax(logits, dim=1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(prob, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1). here for predicting we are inputting the entire prediction
        
        return idx

In [ ]:
# so far we taken these indices, idx and encoded them based on the identity of tokens inside idx. 
# now we are not only encoding identity, but also their posistions
m = BigramLanguageModel()
loss, logits = m(xb,yb)

In [ ]:
# training the model
optimizer = torch.optim.AdamW(m.parameters(),lr=1e-3)
for steps in range(max_iters):
    # every once in a while evaluate loss on train and val
    if steps % eval_interval == 0:
        losses = estimate_loss()
        print(f"step: {steps} -> train loss: {losses['train']: .4f}, val loss: {losses['val']: .4f}") # less noisy loss
    
    # sample batch of data
    xb, yb = get_batch('train')

    # evaluating loss
    loss, logits = m(xb,yb) # here the loss is noisy as every batch will be more/less lucky
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step() # updating parameters
    
# print(loss.item())